# 🤖 模型微调参数配置与训练策略指南（Transformers + Seq2SeqTrainer）


## 🎯 1. 微调方法总览

### ✅ 参数微调（Fine-tuning with hyperparameters）

微调通常是在已有预训练模型上，使用新任务的数据对其参数进行更新。
目标是保留预训练知识的同时学习新任务特征。

**关键参数包括：**
- `learning_rate`
- `num_train_epochs`
- `batch_size`
- `weight_decay`
- `gradient_accumulation_steps`



## 📊 2. 实时跟踪日志 & 训练监控（Logging and Monitoring）

建议使用 `TensorBoard` 或 `WandB` 实时查看训练损失（loss）、验证指标、学习率变化等。

```python
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./out",
    logging_dir="./logs",
    logging_steps=100,
    report_to="tensorboard",  # or 'wandb'
    evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)
```



## 🔁 3. 初始采用默认参数，结合 K-Fold 慢慢调参

初始可采用默认参数快速跑一轮训练（或只用一折），查看训练和验证趋势。

调参建议顺序：
1. 学习率 `learning_rate`
2. 批大小 `batch_size` / 梯度累积 `gradient_accumulation_steps`
3. 保存/评估频率 `save_steps` / `eval_steps`
4. 权重衰减 `weight_decay`



## ⚙️ 4. 动态调整 batch-size 和 step-size

```python
training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2  # 等效于8个样本的批量
)
```

- 显存不足时：减小 `batch_size` + 增大 `gradient_accumulation_steps`
- 显存充足时：可适当增大 `batch_size` 提升训练效率



## ❓ 5. 将每个问题作为单独训练样本（用于问答模型）

适用于 Donut / T5 / GPT 类模型结构化问答训练。

```json
{
  "question": "发票号码是什么？",
  "context": "图像或文本内容",
  "answer": "INV-2025-001"
}
```



## 🧱 6. 过拟合对策（Overfitting）

| 方法 | 说明 |
|------|------|
| 加强正则化 | 提高 `weight_decay`（如从 0.01 → 0.05） |
| 提前停止训练 | 使用 `early stopping` |
| 数据增强 | 如OCR中旋转、模糊图像等 |
| 减小模型容量 | 用 `donut-base` 替代 `donut-large` |
| 增加训练数据 | 提高泛化能力的最好方式 |



## 🧱 7. 欠拟合对策（Underfitting）

| 方法 | 说明 |
|------|------|
| 增大学习率 | 从 `1e-5` → `5e-5` |
| 增加训练轮数 | 3 → 10 |
| 更大模型 | 从 `donut-small` → `donut-base` |
| 减小正则化 | 减少 `weight_decay` |
| 检查数据质量 | 输入可能被截断/压缩 |



## 🖥️ 8. 根据硬件配置参数

| 情况 | 建议 |
|------|------|
| GPU显存小（<8GB） | 降低 `batch_size`，用 `gradient_accumulation_steps` |
| 无GPU / CPU训练 | 设置 `no_cuda=True`，减少轮数 |
| 多卡训练 | 使用 `accelerate` 或 `deepspeed` |
| 显存溢出 | 开启 `fp16=True`（混合精度） |



## ✅ 总结建议

| 阶段 | 要点 |
|------|------|
| 准备阶段 | 数据格式 & 模型选择正确 |
| 初步试验 | 用小数据、小轮次跑通 |
| 参数调整 | 优先调 `learning_rate` 和 `batch_size` |
| 验证监控 | 日志、early stop、tensorboard |
| 过拟合控制 | `weight_decay`、早停、数据增强 |
| 欠拟合修复 | 更多轮次、学习率、模型大小 |
| 资源限制应对 | 梯度累积、混合精度、动态 batch |
